In [2]:
import tensorflow as tf


r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    while True:
      try:
        print(sess.run(next_row))
      except tf.errors.OutOfRangeError:
        break

[ 0.20197356  0.7092995  -2.1252296 ]
[-0.41481382  1.1338934   0.8449984 ]
[ 2.260889   0.7996664 -0.6969561]
[-0.73388064  0.78619623  0.07277289]
[-0.69036645  0.77117145 -0.15173902]
[ 0.26614916 -0.76361793 -0.2721738 ]
[ 2.6574852 -0.6684966 -0.72498  ]
[0.5169061  1.432935   0.41359648]
[-0.04046367  1.0941156  -1.1768993 ]
[ 0.742834   -0.31098646 -0.92690706]


In [7]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

iterator = tf.data.Dataset.from_tensor_slices(my_data)
next_item = iterator.make_one_shot_iterator().get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_item))
        except tf.errors.OutOfRangeError:
            break

[0 1]
[2 3]
[4 5]
[6 7]


In [19]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, lossV = sess.run((optimizer, loss))
  print(lossV)

print(sess.run(y_pred))

sess.close()

0.1716655
0.16825095
0.16558784
0.16344802
0.16167292
0.1601527
0.15881105
0.157595
0.1564678
0.15540397
0.1543858
0.15340094
0.15244092
0.15149972
0.15057331
0.14965871
0.14875399
0.14785776
0.14696893
0.1460869
0.14521116
0.14434138
0.14347728
0.14261869
0.14176545
0.14091747
0.1400747
0.13923702
0.13840443
0.13757683
0.13675424
0.13593656
0.13512379
0.13431594
0.13351285
0.13271458
0.13192111
0.13113236
0.13034834
0.12956898
0.12879431
0.12802427
0.12725882
0.12649798
0.12574165
0.1249899
0.12424259
0.12349976
0.1227614
0.12202741
0.12129781
0.12057261
0.11985171
0.11913515
0.11842282
0.11771481
0.117010996
0.11631141
0.11561602
0.114924766
0.114237666
0.11355466
0.112875715
0.11220087
0.11153003
0.11086321
0.110200375
0.109541476
0.10888657
0.10823558
0.107588455
0.10694518
0.10630578
0.10567018
0.10503839
0.10441038
0.10378613
0.10316561
0.10254884
0.10193571
0.10132624
0.100720435
0.10011826
0.09951964
0.09892464
0.09833317
0.09774527
0.097160846
0.09657994
0.09600249
0.09542851


## Naming

In [ ]:
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
  c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
  with tf.name_scope("inner"):
    c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
  c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
  with tf.name_scope("inner"):
    c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

## Selecting Devices & Remote Session

In [43]:
# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = tf.random_normal([1, 2])

with tf.device("/device:CPU:0"):
  # Operations created in this context will be pinned to the CPU.
  a = tf.constant("Hello TF")

In [ ]:
with tf.device("/device:GPU:0"):
  # Operations created in this context will be pinned to the GPU.
    b = tf.constant("Hello TF with GPU")

In [46]:
# Create a remote session.
with tf.Session("grpc://example.org:2222"):
  pass

## Default Graphs

In [47]:
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

#### Rank

In [48]:
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color

r = tf.rank(my_image)
# After the graph runs, r will hold the value 4.

#### Evaluationg Tensors

In [49]:
p = tf.placeholder(tf.float32)
t = p + 1.0

with tf.Session() as sess:
    t.eval(feed_dict={p:2.0})

InvalidArgumentError: Cannot assign a device for operation Const_5: node Const_5 (defined at <ipython-input-25-ae1fcade62be>:3) was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device. The requested device appears to be a GPU, but CUDA is not enabled.
	 [[node Const_5 (defined at <ipython-input-25-ae1fcade62be>:3) ]]

Caused by op 'Const_5', defined at:
  File "/Users/mah/miniconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mah/miniconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mah/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mah/miniconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mah/miniconda3/lib/